In [1]:
%%javascript

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-k', {
    help : 'move up selected cells',
    help_index : 'jupyter-notebook:move-selection-up',
    handler : function (event) {
        IPython.notebook.move_selection_up();
        return false;
    }}
);

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-j', {
    help : 'move down selected cells',
    help_index : 'jupyter-notebook:move-selection-down',
    handler :  function (event) {
        IPython.notebook.move_selection_down();
        return false;
    }}
);

<IPython.core.display.Javascript object>

# Feature documentation

## Features Actuales

   ###  Model CountVector: 
    Utilice CountVectorizer de sklearn y usando random forest me quede con los 10 features mas importantes
    
   ### Feature Most Viewed and Searched 
    Me quede con el numero de vistos (en este caso tuve encuenta el modelo) y buscado mas grande en un dia 
    
   ### Lead and conversion
    Cantidad de lead eventos que realizo y cantidad de conversions que realizo
    
   ### Most common searched values in conversion persons count vector
    Las busquedas mas comunes usando countVector()
    
   ### Diff caro
       Magia negra
    
## Features probados que no funcionaron y otras cosas que tampoco:

   ### Cantidad de eventos por tipo
       Numero de veces que realizo cada evento en dataframe
   ### Algoritmo 0.63 con RandomFores
        Use random forest para ver la importancia de lo features de las busquedas y de los modelos y me quede con los mejores 10, el resultado fue menor con 0.61
   
## Best Score: 0.63897 
     - commit 0.63 
     - features 
         - model countvector (sin random forest seleccionando un top)
         - most viewed and searched
         - lead and conversion
         - most common search (sin random forest seleccionando un top)
         - diff caro
               
## Cosas a probar:
    -Kaggle score de lo hecho con random forest
    -
 
 
 
## Acotaciones
    -random forest en search y model, lo que hice fue tomar los features por separado y ver sus importancias
               
        
     

In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from scipy.sparse import hstack
from xgboost.sklearn import XGBClassifier # <3
from sklearn.model_selection import train_test_split
import gc
import matplotlib.pyplot as plt

from scipy.sparse.csr import csr_matrix

pd.options.mode.chained_assignment = None


In [3]:
df = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)


In [4]:
df_labels_one = df_labels.loc[df_labels['label']==1]
df_labels_one = df.merge(df_labels_one, on='person', how='right')

In [5]:
subjects = df['person'].drop_duplicates().to_frame()
subjects.shape

(38829, 1)

In [6]:
df['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

## Modelo count vector

TOP 10 FEATURES ['5s', '6s', 'edge', 'galaxy', 'iphone', 'plus', 's7', 's8', 'samsung', 'se']


try : list of values that exist in conversion

In [7]:
marcas = ['iphone' , 'samsung', 'lg', 'motorola', 'sony', 'asus', 'lenovo']

In [8]:
#df_all = df_labels.merge(df_model, left_on = 'person', right_on = 'person', how = 'left')
#df_all.head()

In [9]:
df_model = df.loc[df['event'] == 'viewed product']
df_model = df_model.groupby('person')['model'].apply(list).reset_index()
df_model.head()

,person,model
0,00091926,"[iPhone 6 Plus, iPhone 6S, iPhone 6S, Motorola..."
1,00091a7a,"[iPhone SE, iPhone 6, iPhone 6S]"
2,000ba417,"[Samsung Galaxy A3 2016, Samsung Galaxy Gran P..."
3,000c79fe,"[iPhone 7, iPhone 7, iPhone 7]"
4,000e4d9e,"[Samsung Galaxy S4 i9505, Samsung Galaxy S6 Fl..."


In [10]:
df_model['cantidad_vistos'] = df_model['model'].apply(len)
df_model['model'] = df_model['model'].apply(lambda x: ', '.join(map(str, x)))
df_model = df_model.fillna('')
df_model.head()

,person,model,cantidad_vistos
0,00091926,"iPhone 6 Plus, iPhone 6S, iPhone 6S, Motorola ...",372
1,00091a7a,"iPhone SE, iPhone 6, iPhone 6S",3
2,000ba417,"Samsung Galaxy A3 2016, Samsung Galaxy Gran Pr...",153
3,000c79fe,"iPhone 7, iPhone 7, iPhone 7",3
4,000e4d9e,"Samsung Galaxy S4 i9505, Samsung Galaxy S6 Fla...",339


In [11]:
word_vectorizer = CountVectorizer(vocabulary = marcas)

tf_mat = word_vectorizer.fit_transform(df_model['model'])

tf_array = tf_mat.toarray()
tf_array.shape

(37130, 7)

In [12]:
feature_names = word_vectorizer.get_feature_names()

In [13]:
df_tfidf = pd.DataFrame(tf_array, columns=feature_names)
df_tfidf['person'] = df_model['person']
df_tfidf['cantidad_vistos'] = df_model['cantidad_vistos']

In [14]:
df_top = df_model['cantidad_vistos'].to_frame()
df_top['person'] = df_model['person']

In [15]:
df_top.shape

(37130, 2)

In [16]:
df_top.head()

,cantidad_vistos,person
0,372,00091926
1,3,00091a7a
2,153,000ba417
3,3,000c79fe
4,339,000e4d9e


In [17]:
df_top = subjects.merge(df_top, on = 'person', how='left')

In [ ]:
df_top = 

## Diff de lo de caro 0.62

In [18]:
df_diff = pd.read_csv('diff.csv')
df_top = df_top.merge(df_diff , left_on='person', right_on='person', how='left')
df_top.shape

(38829, 5)

##  Feature Most Viewed and Searched in a day

In [19]:
df_v = df.loc[df['event'] == 'viewed product']
df_v['timestamp'] = pd.to_datetime(df_v['timestamp'])
df_v['count'] = 1
df_v['month'] = df_v['timestamp'].dt.month
df_v['day'] = df_v['timestamp'].dt.day

df_v = df_v.groupby(['person','month', 'day']).agg({'count':'sum'})
df_v = df_v.groupby(['person']).agg({'count':'mean'}).reset_index()
df_v.head()

,person,count
0,00091926,18.600000
1,00091a7a,3.000000
2,000ba417,51.000000
3,000c79fe,3.000000
4,000e4d9e,37.666667


In [20]:
df_s = df.loc[df['event'] == 'searched products']
df_s['timestamp'] = pd.to_datetime(df_s['timestamp'])
df_s['count'] = 1
df_s['month'] = df_s['timestamp'].dt.month
df_s['day'] = df_s['timestamp'].dt.day

df_s = df_s.groupby(['person','month', 'day']).agg({'count':'sum'})
df_s = df_s.groupby(['person']).agg({'count':'mean'}).reset_index()
df_s['person'].describe()

count        13093
unique       13093
top       234ceb64
freq             1
Name: person, dtype: object

In [21]:
df_s.columns = ['person','searched']
df_v.columns = ['person', 'viewed']
df_s.head()

,person,searched
0,000c79fe,9.000000
1,000e619d,3.000000
2,001001be,17.000000
3,001802e4,4.000000
4,0019e639,3.666667


In [22]:
df_vs_feat = df_v.merge(df_s,  on='person', how='left')
df_vs_feat.head()

,person,viewed,searched
0,00091926,18.600000,NaN
1,00091a7a,3.000000,NaN
2,000ba417,51.000000,NaN
3,000c79fe,3.000000,9.0
4,000e4d9e,37.666667,NaN


In [23]:
df_top = df_top.merge(df_vs_feat, left_on='person', right_on='person', how='left')
df_top.shape

(38829, 7)

In [24]:
df_top.head()

,person,cantidad_vistos,diff_mean,diff_max,session_count,viewed,searched
0,4886f805,4.0,0.058056,0.058056,1.0,4.000000,1.0
1,ad93850f,20.0,0.095324,0.467500,6.0,6.666667,NaN
2,0297fc1e,404.0,0.120095,0.678889,70.0,7.214286,1.5
3,2d681dd8,13.0,0.170093,0.376389,3.0,6.500000,1.0
4,cccea85e,739.0,0.152070,0.847778,42.0,56.846154,1.0


## Lead and conversion

In [25]:
df_lead = df.loc[df['event'] == 'lead']
df_lead = df_lead['person'].drop_duplicates().to_frame()
df_lead['lead'] = 1

df_conversion = df.loc[df['event'] == 'conversion']
df_conversion = df_conversion['person'].drop_duplicates().to_frame()
df_conversion['conversion'] = 1


In [26]:
df_top = df_top.merge(df_lead, left_on='person', right_on='person', how='left')
df_top = df_top.merge(df_conversion, left_on='person', right_on='person', how='left')

df_top.shape

(38829, 9)

## Most common searched values in conversion persons count vector
TOP 10 CON RF = ['5s', '6s', 'galaxy', 'iphone', 'j5', 'j7', 'moto', 'plus', 's8', 'samsung']

In [27]:
person_conv = (df.loc[df['event']=='conversion'])['person'].drop_duplicates()
df_conv = df.loc[df['person'].isin(person_conv)]

In [28]:
df_search = df.loc[df['search_term'].notnull()]
df_search = df_search.groupby('person')['search_term'].apply(list).reset_index()

In [29]:
#drop nans
df_search.head()

,person,search_term
0,000c79fe,"[Iphone 7, Galaxy a8, Iphone 7, Galaxy s8, Iph..."
1,000e619d,"[samsung rosa, sansung j7, sansung j7, sansung..."
2,001001be,"[IPhone 6, 5s, IPhone 6, IPhone 6, 5s, IPho..."
3,001802e4,"[Aiphone 6s, Aiphone 6s]"
4,0019e639,"[ON 7, ON 7, ON 7, ON 7, ON 7, ON 7, ON 7, ON ..."


In [30]:
#df_all = df_labels.merge(df_search, left_on = 'person', right_on = 'person', how = 'left')
#df_all.head()

In [31]:
df_search['search_term'] = df_search['search_term'].apply(lambda x: ', '.join(map(str, x)))
df_search = df_search.fillna('')
df_search.shape

(12570, 2)

In [32]:
word_vectorizer = CountVectorizer(vocabulary=marcas)

tf_mat = word_vectorizer.fit_transform(df_search['search_term'])

tf_array = tf_mat.toarray()
tf_array.shape

(12570, 7)

In [33]:
doc = 0
feature_names = word_vectorizer.get_feature_names()

In [34]:
df_countvect = pd.DataFrame(tf_array, columns=feature_names)
df_countvect['person'] = df_search['person']
df_countvect.head()

,iphone,samsung,lg,motorola,sony,asus,lenovo,person
0,6,0,0,0,0,0,0,000c79fe
1,0,1,0,0,0,0,0,000e619d
2,12,0,0,0,0,0,0,001001be
3,0,0,0,0,0,0,0,001802e4
4,0,1,0,0,0,0,0,0019e639


In [35]:
df_countvect['person'] = df_search['person']
df_countvect.shape

(12570, 8)

In [36]:
df_top = df_top.merge(df_countvect, left_on='person', right_on='person', how='left')

In [37]:

df_top.shape

(38829, 16)

## XGboost

In [38]:
df_train = df_labels.merge(df_top , left_on='person', right_on='person', how='inner')

In [39]:
df_train.shape

(19414, 17)

In [40]:
df_train.head()

,person,label,cantidad_vistos,diff_mean,diff_max,session_count,viewed,searched,lead,conversion,iphone,samsung,lg,motorola,sony,asus,lenovo
0,0566e9c1,0,23.0,0.059618,0.247778,8.0,11.500000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6ec7ee77,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abe7a2fb,0,31.0,0.095119,0.346944,21.0,3.444444,2.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34728364,0,24.0,0.043148,0.080556,3.0,8.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,87ed62de,0,9.0,0.155278,0.172222,2.0,4.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X = X.fillna(0)
X.head()

,cantidad_vistos,diff_mean,diff_max,session_count,viewed,searched,lead,conversion,iphone,samsung,lg,motorola,sony,asus,lenovo
0,23.0,0.059618,0.247778,8.0,11.500000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31.0,0.095119,0.346944,21.0,3.444444,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,24.0,0.043148,0.080556,3.0,8.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9.0,0.155278,0.172222,2.0,4.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
import xgboost as xgb
from sklearn.linear_model import LinearRegression

In [92]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [93]:
reg = LinearRegression().fit(X, y)

In [94]:
persons = df_labels['person']
df_predict = df_top.loc[~df_top.person.isin(persons)]
ppl_to_predict = (df.loc[~df['person'].isin(persons)])['person'].to_frame()
ppl_to_predict = ppl_to_predict.drop_duplicates('person')
df_predict = ppl_to_predict.merge(df_predict, left_on = 'person' , right_on = 'person', how='left')
X_predict = df_predict.drop(['person'], axis=1)

In [95]:
X_predict = X_predict.fillna(0)

In [96]:
entrie = reg.predict(X_predict)

In [97]:
seriesita = pd.Series(entrie)

In [98]:
df_entrie = df_predict['person'].to_frame()
df_entrie['label'] = seriesita

In [99]:
df_entrie.head()

,person,label
0,4886f805,0.037845
1,0297fc1e,0.159985
2,2d681dd8,0.045270
3,cccea85e,0.090473
4,4c8a8b93,0.080491


In [109]:
df_entrie = df_entrie.fillna(0)

In [110]:
num = df_entrie._get_numeric_data()
num[num > 1] = 1

In [118]:
df_entrie.to_csv(path_or_buf = 'linear_reg.csv', index = False)

In [112]:
df_entrie.shape

(19415, 2)

In [113]:
df_entrie['label'].nlargest()

47       1.000000
4309     0.449890
197      0.384200
9323     0.364062
13217    0.322093
Name: label, dtype: float64

## Scoring

Para evaluar usen esta medida que me da valores muy parecidos a los de kaggle, para hacer las predicciones usen el otro

In [114]:
my_classifier1 = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 6,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 10)

In [115]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [116]:
my_classifier1.fit(X_train,y_train)
entrie = my_classifier1.predict_proba(X_predict)

In [117]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  my_classifier1.predict_proba(X_test)[:,1])

0.6252616382953379

 # Random Forest feature importance
    -TEST THIS

In [115]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
#Load boston housing dataset as an example

names = X.columns
rf = RandomForestRegressor()
rf.fit(X, y)
print ("Features sorted by their score:")
zipped = zip(map(lambda x: round(x, 4), rf.feature_importances_), names)
feature = sorted(zipped, key=lambda x: x[1])

/home/amaherok/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Features sorted by their score:


In [116]:
X.shape

(19414, 6)

In [120]:
feat_importance = pd.DataFrame(feature, columns=['importance', 'feature'])
feat_importance

,importance,feature
0,0.1831,cantidad_vistos
1,0.0153,conversion
2,0.5295,diff
3,0.0072,lead
4,0.1036,searched
5,0.1613,viewed


In [118]:
feat_importance =feat_importance.loc[feat_importance['importance'].isin(feat_importance['importance'].nlargest(100))]
feats_servibles = feat_importance['feature'].tolist()
feats_servibles

['cantidad_vistos', 'conversion', 'diff', 'lead', 'searched', 'viewed']

In [119]:
df_top = df_top[feats_servibles]
df_top['person'] = subjects
df_top.head()

,cantidad_vistos,conversion,diff,lead,searched,viewed,person
0,4.0,NaN,0.058056,NaN,1.0,4.0,4886f805
1,20.0,NaN,0.095324,NaN,NaN,15.0,ad93850f
2,404.0,NaN,0.120095,1.0,2.0,26.0,0297fc1e
3,13.0,NaN,0.170093,NaN,1.0,7.0,2d681dd8
4,739.0,NaN,0.152070,NaN,1.0,94.0,cccea85e


In [121]:
feats_servibles

['cantidad_vistos', 'conversion', 'diff', 'lead', 'searched', 'viewed']

In [146]:
df_v = df.loc[df['event'] == 'viewed product']
df_v['timestamp'] = pd.to_datetime(df_v['timestamp'])
df_v['count'] = 1
df_v['month'] = df_v['timestamp'].dt.month
df_v['day'] = df_v['timestamp'].dt.day

df_v = df_v.groupby(['person','month', 'day','model']).agg({'count':'sum'})
#df_v = df_v.sort_values(by=['person','count'],ascending=False ).groupby('person').head(2).reset_index()

df_v = df_v.groupby(['person']).agg({'count':'mean'}).reset_index()
df_v.head()

,person,count
0,00091926,4.133333
1,00091a7a,1.000000
2,000ba417,4.371429
3,000c79fe,3.000000
4,000e4d9e,4.985294


In [147]:
df_s = df.loc[df['event'] == 'searched products']
df_s['timestamp'] = pd.to_datetime(df_s['timestamp'])
df_s['count'] = 1
df_s['month'] = df_s['timestamp'].dt.month
df_s['day'] = df_s['timestamp'].dt.day

df_s = df_s.groupby(['person','month', 'day', 'search_term']).agg({'count':'sum'})
#df_s = df_s.sort_values(by=['person','count'], ascending=False).groupby('person').head(2).reset_index()
df_s = df_s.groupby(['person']).agg({'count':'mean'}).reset_index()
df_s.head()

,person,count
0,000c79fe,1.142857
1,000e619d,3.000000
2,001001be,8.000000
3,001802e4,2.000000
4,0019e639,2.200000


In [148]:
df_s.columns = ['person','searched']
df_v.columns = ['person', 'viewed']
df_s.head()

,person,searched
0,000c79fe,1.142857
1,000e619d,3.000000
2,001001be,8.000000
3,001802e4,2.000000
4,0019e639,2.200000


In [149]:
df_vs_feat = df_v.merge(df_s,  on='person', how='left')
df_vs_feat.head()

,person,viewed,searched
0,00091926,4.133333,NaN
1,00091a7a,1.000000,NaN
2,000ba417,4.371429,NaN
3,000c79fe,3.000000,1.142857
4,000e4d9e,4.985294,NaN


In [270]:
df_top = df_top.merge(df_vs_feat, left_on='person', right_on='person', how='left')
df_top.shape

(38829, 3)